In [1]:
%matplotlib inline

import os

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn import linear_model, datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

# distributions
from scipy.stats import randint 
from scipy.stats import uniform

#Bayessearch cv uit Scikit-optimize
from skopt import BayesSearchCV


from sklearn.svm import SVC
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
# Import Keras libraries

# Import Tensorflow libraries
import keras
from keras.models import Sequential
from keras.layers import Activation
from keras.optimizers import SGD
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.layers import MaxPooling2D
from keras import backend as K
from keras.utils import np_utils

Using TensorFlow backend.


In [2]:
# Import dataset
dataset = pd.read_csv('cancer.csv')
dataset.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [3]:
# Data preprocessing
dataset.drop('id',axis=1, inplace=True)

# Remove right column (unnamed)
dataset.drop(dataset.columns[31],axis=1,inplace=True)

# diagnosis labels M -> 1 and B -> 0
dataset.diagnosis.replace(['M', 'B'], [1, 0], inplace=True)

In [4]:
# Split dataset up in features and targets
y = dataset.diagnosis
X = dataset.drop('diagnosis',axis=1)

In [14]:
# Create training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state =0)

# feature normalization
# via robustscaler: This Scaler removes the median and scales the data according to the quantile range 

Rscaler = RobustScaler()
Rscaler.fit(X_train)

X_train = Rscaler.transform(X_train)
X_test = Rscaler. transform(X_test)


In [15]:
# Train an SVM (grid search)

model = SVC()
parameters = [ 
        {'kernel': ['linear'], 'C': np.linspace(1,20,10)},
        {'kernel': ['rbf'], 'C': [1, 10], 'gamma': [0.0001, 0.001, 0.01, 0.1, 0.2]},
        {'kernel': ['poly'], 'C':[1, 10]} ]
grid_search = GridSearchCV(estimator = model, 
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 4,
                           n_jobs = -1,
                           verbose = 3)
grid_search = grid_search.fit(X_train, y_train)

best_accuracy = grid_search.best_score_ 
best_parameters = grid_search.best_params_  

print('Best accuracy : ', grid_search.best_score_)
print('Best parameters :', grid_search.best_params_  )

y_pred = grid_search.predict(X_test)
print(classification_report(y_test, y_pred))

cf = confusion_matrix(y_test, y_pred)
print(cf)
print(accuracy_score(y_test, y_pred) * 100) 




Fitting 4 folds for each of 22 candidates, totalling 88 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    2.0s


Best accuracy :  0.9859154929577465
Best parameters : {'C': 1.0, 'kernel': 'linear'}
              precision    recall  f1-score   support

           0       0.98      0.96      0.97        90
           1       0.93      0.96      0.94        53

   micro avg       0.96      0.96      0.96       143
   macro avg       0.95      0.96      0.96       143
weighted avg       0.96      0.96      0.96       143

[[86  4]
 [ 2 51]]
95.8041958041958


[Parallel(n_jobs=-1)]: Done  65 out of  88 | elapsed:    2.3s remaining:    0.8s
[Parallel(n_jobs=-1)]: Done  88 out of  88 | elapsed:    2.4s finished
/home/wouter/tensorflow/lib/python3.6/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [16]:
# Train an SVM (random search)

model = SVC()
parameters = {'kernel': ['linear','rbf','poly'],
              'C': uniform(0.01, 100), 
              'gamma': uniform(0.001, 2)}


n_iter_search = 40


random_search = RandomizedSearchCV(model, param_distributions=parameters,cv=4,n_iter=n_iter_search)

random_search = random_search.fit(X_train, y_train)

best_accuracy = random_search.best_score_ 
best_parameters = random_search.best_params_  

print('Best accuracy : ', random_search.best_score_)
print('Best parameters :',random_search.best_params_  )

y_pred = random_search.predict(X_test)
print(classification_report(y_test, y_pred))

cf = confusion_matrix(y_test, y_pred)
print(cf)
print(accuracy_score(y_test, y_pred) * 100) 




Best accuracy :  0.9835680751173709
Best parameters : {'C': 1.7566526744992272, 'gamma': 0.4914672460054458, 'kernel': 'linear'}
              precision    recall  f1-score   support

           0       0.97      0.98      0.97        90
           1       0.96      0.94      0.95        53

   micro avg       0.97      0.97      0.97       143
   macro avg       0.96      0.96      0.96       143
weighted avg       0.96      0.97      0.96       143

[[88  2]
 [ 3 50]]
96.5034965034965


/home/wouter/tensorflow/lib/python3.6/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [8]:
# Train an SVM (Bayes optimization )

model = SVC()
parameters = {'kernel': ['linear','rbf','poly'],
              'C': (0.01, 100,'uniform'), # 
              'gamma': (0.001, 2,'uniform')}

n_iter_search = 20

Bayes_search = BayesSearchCV(model,parameters,n_iter=n_iter_search,cv=4,verbose=1)


Bayes_search.fit(X_train, y_train)

best_accuracy = Bayes_search.best_score_ 
best_parameters = Bayes_search.best_params_  

print('Best accuracy : ', Bayes_search.best_score_)
print('Best parameters :',Bayes_search.best_params_  )

y_pred = Bayes_search.predict(X_test)
print(classification_report(y_test, y_pred))

cf = confusion_matrix(y_test, y_pred)
print(cf)
print(accuracy_score(y_test, y_pred) * 100) 



Fitting 4 folds for each of 1 candidates, totalling 4 fits
Fitting 4 folds for each of 1 candidates, totalling 4 fits
Fitting 4 folds for each of 1 candidates, totalling 4 fits
Fitting 4 folds for each of 1 candidates, totalling 4 fits
Fitting 4 folds for each of 1 candidates, totalling 4 fits
Fitting 4 folds for each of 1 candidates, totalling 4 fits
Fitting 4 folds for each of 1 candidates, totalling 4 fits
Fitting 4 folds for each of 1 candidates, totalling 4 fits
Fitting 4 folds for each of 1 candidates, totalling 4 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_j

Fitting 4 folds for each of 1 candidates, totalling 4 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s finished


Fitting 4 folds for each of 1 candidates, totalling 4 fits


...

Fitting 4 folds for each of 1 candidates, totalling 4 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s finished
/home/wouter/tensorflow/lib/python3.6/site-packages/skopt/optimizer/optimizer.py:399: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Fitting 4 folds for each of 1 candidates, totalling 4 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s finished


Best accuracy :  0.9835680751173709
Best parameters : {'C': 100.0, 'gamma': 0.001, 'kernel': 'rbf'}
              precision    recall  f1-score   support

           0       0.98      0.99      0.98        90
           1       0.98      0.96      0.97        53

   micro avg       0.98      0.98      0.98       143
   macro avg       0.98      0.98      0.98       143
weighted avg       0.98      0.98      0.98       143

[[89  1]
 [ 2 51]]
97.9020979020979


# Neural Network hyperparametertuning

In [17]:
# Import dataset
dataset = pd.read_csv('cancer.csv')
dataset.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [18]:
# Data preprocessing
dataset.drop('id',axis=1, inplace=True)

# Remove right column (unnamed)
dataset.drop(dataset.columns[31],axis=1,inplace=True)

# diagnosis labels M -> 1 and B -> 0
dataset.diagnosis.replace(['M', 'B'], [1, 0], inplace=True)


# Split dataset up in features and targets
y = dataset.diagnosis
X = dataset.drop('diagnosis',axis=1)



In [19]:
# Create training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state =0)

# Convert to numpy array
X_train = X_train.values
X_test = X_test.values

# One-hot encoding of the targets
y_train = np_utils.to_categorical(y_train)

# Feature normalization
# via robustscaler: This Scaler removes the median and scales the data according to the quantile range 

Rscaler = RobustScaler()
Rscaler.fit(X_train)

X_train = Rscaler.transform(X_train)
X_test = Rscaler. transform(X_test)

In [20]:
def create_model(optimizer='adam',activation = 'relu',dropout_rate = 0.0, kernel_initializer='uniform',neurons = 10):
    model = Sequential()
    model.add(Dense(neurons, input_dim=X_train.shape[1], kernel_initializer=kernel_initializer,activation="relu"))
    model.add(Dropout(dropout_rate))
    model.add(BatchNormalization())
    model.add(Dense(neurons, kernel_initializer=kernel_initializer,activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(BatchNormalization())
    model.add(Dense(neurons, kernel_initializer=kernel_initializer,activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(BatchNormalization())
    model.add(Dense(2, activation='softmax'))

    model.compile(loss=keras.losses.categorical_crossentropy,optimizer=optimizer,metrics=['accuracy']) 
    return model

In [ ]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV


model = KerasClassifier(build_fn=create_model, batch_size=8, epochs =10)

input_dim= [X_train.shape[1]]
activation =  ['relu', 'tanh', 'sigmoid'] 
#learn_rate = [0.001, 0.01, 0.1, 0.2, 0.3]
dropout_rate = [0.0, 0.1, 0.5]
#weight_constraint=[1, 2, 3, 4, 5]
neurons = [5, 10, 20, 30, 40 ,50]
init = ['uniform', 'lecun_uniform', 'normal']
optimizer = [ 'SGD', 'adam']
#optimizer = [ 'SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']

epochs = [50] 
batch_size = [8, 16, 32] 
param_grid = dict(epochs=epochs, batch_size=batch_size,optimizer = optimizer, dropout_rate = dropout_rate,activation = activation)
##############################################################
grid = GridSearchCV(estimator=model, param_grid = param_grid,verbose=3)
grid_result = grid.fit(X_train, y_train) 



/home/wouter/tensorflow/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 3 folds for each of 54 candidates, totalling 162 fits
[CV] activation=relu, batch_size=8, dropout_rate=0.0, epochs=50, optimizer=SGD 
Epoch 1/50
284/284 [==============================] - 9s 31ms/step - loss: 0.3610 - acc: 0.8732
Epoch 2/50
284/284 [==============================] - 0s 1ms/step - loss: 0.2943 - acc: 0.9014
Epoch 3/50
284/284 [==============================] - 0s 2ms/step - loss: 0.2869 - acc: 0.9085
Epoch 4/50
284/284 [==============================] - 0s 1ms/step - loss: 0.1638 - acc: 0.9648
Epoch 5/50
284/284 [==============================] - 0s 981us/step - loss: 0.2582 - acc: 0.9049
Epoch 6/50
284/284 [==============================] - 0s 507us/step - loss: 0.1438 - acc: 0.9577
Epoch 7/50
284/284 [==============================] - 0s 1ms/step - loss: 0.2081 - acc: 0.9401
Epoch 8/50
284/284 [==============================] - 0s 2ms/step - loss: 0.2058 - acc: 0.9331
Epoch 9/50
284/284 [==============================] - 0s 2ms/step - loss: 0.1837 - acc: 0.950

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   28.5s remaining:    0.0s


Epoch 1/50
284/284 [==============================] - 9s 32ms/step - loss: 0.3736 - acc: 0.8169
Epoch 2/50
284/284 [==============================] - 0s 1ms/step - loss: 0.3552 - acc: 0.8697
Epoch 3/50
284/284 [==============================] - 0s 1ms/step - loss: 0.2377 - acc: 0.9190
Epoch 4/50
284/284 [==============================] - 0s 1ms/step - loss: 0.2103 - acc: 0.9261
Epoch 5/50
284/284 [==============================] - 0s 1ms/step - loss: 0.1531 - acc: 0.9437
Epoch 6/50
284/284 [==============================] - 0s 1ms/step - loss: 0.1502 - acc: 0.9437
Epoch 7/50
284/284 [==============================] - 0s 1ms/step - loss: 0.0982 - acc: 0.9789
Epoch 8/50
284/284 [==============================] - 0s 1ms/step - loss: 0.2349 - acc: 0.9120
Epoch 9/50
284/284 [==============================] - 0s 1ms/step - loss: 0.1601 - acc: 0.9613
Epoch 10/50
284/284 [==============================] - 0s 1ms/step - loss: 0.1178 - acc: 0.9577
Epoch 11/50
284/284 [===========================

[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   59.7s remaining:    0.0s


Epoch 1/50


In [ ]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))